<a href="https://colab.research.google.com/github/amand111/colab/blob/main/Rag_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install langchain
!pip install openai
!pip install unstructured
!pip install tiktoken
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 28.0 MB/s eta 0:00:00


In [24]:
# prompt: add env variable OPENAI_API_KEY

!export OPENAI_API_KEY=""


In [8]:
# RAG-Demo: all imports go here

import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [14]:
#RAG-Demo: Test connection using a simple prompt
import langchain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

template = """Question: {question}
"""
prompt = PromptTemplate.from_template(template)
openai_api_keyvar=""

llm = OpenAI(openai_api_key=openai_api_keyvar)
question = "What does Microsoft do? Who is CEO"

llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain.run(question)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\nMicrosoft is a multinational technology company that develops, manufactures, licenses, supports, and sells computer software, consumer electronics, and personal computers. The company is best known for its Windows operating system and Microsoft Office suite of productivity software.\n\nThe CEO of Microsoft is Satya Nadella, who has held the position since 2014.'

In [19]:
# prompt: loaders unstructuredURLLoader,

from langchain.document_loaders import UnstructuredURLLoader

# Create an UnstructuredURLLoader instance
loaders = UnstructuredURLLoader(urls=[
    "https://en.wikipedia.org/wiki/Microsoft",
    "https://en.wikipedia.org/wiki/Google",
    ])

# Use the loader to load documents from the URL
documents = loaders.load()

# Print the number of documents loaded
print(f"Number of documents loaded: {len(documents)}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Number of documents loaded: 2


In [20]:
# prompt: Splitting text by recursive look at characters

# Create a RecursiveCharacterTextSplitter instance with a character overlap of 40
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=100, chunk_size=900)

# Use the text_splitter to split the documents into chunks
docs = text_splitter.split_documents(documents)

# Print the number of documents after splitting
print(f"Number of documents after splitting: {len(docs)}")


Number of documents after splitting: 393


In [29]:
# prompt: User openAIEmbeddings to create embeddings
from langchain.embeddings import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(openai_api_key="sk-proj-y8ujNvfd0wrgfN8uRcEuT3BlbkFJ3jcLiPerZKFwiB1IIWrj")
# vectorstore = FAISS.from_documents(docs, embeddings)

query_result = embeddings.embed_query("hello how are you doing")
len(query_result)
print(query_result)





[-0.03122585592397714, -0.005931257069220259, -0.006895921966365138, -0.04217782804778852, -0.02832231051911375, 0.02139454989213113, -0.01817263371357744, -0.0090735819513353, -0.021063444212987413, -0.0008018977017256289, 0.02671771801991838, 0.00829675551264446, -0.021305407416602418, -0.0043616864913444174, -0.004972959624802092, -0.021827534814902218, 0.0254060283461517, -0.024298097159639757, 0.011168464585196916, -0.006934126682725402, -0.007679115392121382, -2.6962068043327534e-05, 0.0019452485425218607, -0.013486208064493401, -0.00843047201990538, -0.016402489306132834, 0.01055082446467384, -0.010996544292898347, 0.021993088585796693, -0.0439352375497797, -0.0010362985074897186, 0.010697274946065566, -0.018974929031852508, 0.0009312360031603022, -0.0014223236992319686, -0.016886413850717603, -0.01347347315903998, -0.013689965620425524, 0.02138181498667771, -0.008073895598532206, 0.02112711874025452, -0.014199359975917137, 0.0020423518473582157, -0.006934126682725402, -0.013817

In [33]:
vectorindex_openaiEmbed = FAISS.from_documents(docs, embeddings)

chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorindex_openaiEmbed.as_retriever())

In [38]:
query = "Who is larry"

langchain.debug=False

chain({"question": query}, return_only_outputs=True)

{'answer': ' Larry could potentially be Larry Page, the co-founder of Google. It is unclear without further context. \n',
 'sources': 'https://en.wikipedia.org/wiki/Google'}